<h1 style="padding-top: 25px;padding-bottom: 25px;text-align: left; padding-left: 10px; background-color: #DDDDDD; 
    color: black;"> <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> CS109A Introduction to Data Science </h1>

## Homework 1: Data Collection, Parsing, and Quick Analyses

**Harvard University**<br/>
**Fall 2020**<br/>
**Instructors**: Pavlos Protopapas, Kevin Rader, and Chris Tanner<br/>
<hr style='height:2px'>

In [1]:
## RUN THIS CELL TO GET THE RIGHT FORMATTING 
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2020-CS109A/master/themes/static/css/cs109.css").text
HTML(styles)


## Overview 

In this homework, your goal is to learn how to acquire, parse, clean, and analyze data. Toward this goal, we will address certain question about COVID, and you will scrape data directly from a website. For the remainder of the semester, we will provide you data files directly; however, since real-world problems often require gathering information from a variety of sources, including the Internet, web scraping is a highly useful skill to have.

### Instructions
- To submit your assignment, follow the instructions given in Canvas.

### Learning Objectives
- Get started using Jupyter Notebooks, which are incredibly popular, powerful, and will be our medium of programming for the duration of CS109A and CS109B.
- Become familiar with how to access and use data from various sources (i.e., web scraping and directly from files).
- Gain experience with data exploration and simple analysis.
- Become comfortable with PANDAS as a means of storing and working with data.
- Reflect on what further analysis you may wish to do with this data. For example, given the material we've covered so far, what *more* do you wish you had the ability to do (e.g., modelling, prediction, etc). That is, think about questions you may have about the data, and try to imagine what types of tools you might need to help answer your questions.

### Notes
- Exercise **responsible scraping**. Web servers can become slow or unresponsive if they receive too many requests from the same source in a short amount of time. In your code, use a delay of 2 seconds between requests. This helps to not get blocked by the target website -- imagine how frustrating it would be to have this occur. Section 1 of this homework involves saving the scraped web pages to your local machine. Thus, after completing Section 1, you do not need to re-scrape any of the pages, unless you wish to occasionally grab the latest data. 

- Web scraping requests can take several minutes. This is another reason why you should not wait until the last minute to do this homework.
- As you run a Jupyter Notebook, it maintains a running state of memory. Thus, the order in which you run cells matters and plays a crucial role; it can be easy to make mistakes based on *when* you run different cells as you develop and test your code. **Before submitting every Jupyter Notebook homework assignment, be sure to restart your Jupyter Notebook and run the entire notebook from scratch, all at once (i.e., "Kernel -> Restart & Run All")**
- We will be working with COVID data. COVID has impacted everyone in the world, and naturally some people have been greatly more affected than others. We, the teaching staff, are sensitive to this, empathize, and understand that working with COVID data may be unsettling to some. We apologize for any discomfort this may cause. Our intent with this assignment is purely pedagogical, and we'd like to remind students that data science and machine learning can be used to provide insights that can be used for good and invoke change. Toward this goal, parts of the homework are intended to shed light on the unfortunate, widespread inequality that exists. So, while this data may be unsettling, our aim is for the learned skills addressed here -- and in all future assignments -- to provide you with knowledge and confidence to do good work.

## 1. Obtaining Data (17 points)

For any given situation or scenario that we wish to understand, we will rely on having relevant data. Here, we are interested in the degree to which the SARS-CoV-2 virus has affected United States citizens (SARS-CoV-2 is the virus that causes the COVID-19 disease). The Centers for Disease Control and Prevention (CDC) provides relevant data from USAFacts.org that includes the number of confirmed COVID-19 cases on a per-county basis. Visit https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/. At the bottom of the web page, in a blue table, you should see a list of every state, each of which has its own web page.

In this exercise, we will focus on automating the downloading of each state's data (via ``Requests``). First, as we will do for every Jupyter Notebook, let's import necessary packages that we will use throughout the notebook (i.e., run the cell below). 

In [2]:
# import the necessary libraries
import re
import requests
import pandas as pd
import numpy as np
from time import sleep
from bs4 import BeautifulSoup

# NOTE: files will be saved to this directory, so you need to ensure
# that it exists on your system first (it should be visible from the
# directory of where you are running this Notebook file)
# i.e.,
# >> ls
# cs109a_hw1_student.ipynb
# election2016_by_county.csv
# data/
state_dir = "data/"

In [3]:
# we define this for convenience, as every state's url begins with this prefix
base_url = 'https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/'

<div class='exercise'><b> Exercise 1.1 [1 pt]: Fetching Website data via Requests</b>

Fetch the web page located at `base_url` and save the request's returned object (a Response object) to a variable named `home_page`.
</div>

In [4]:
# YOUR CODE HERE
home_page = requests.get(base_url)
# END OF YOUR CODE HERE

<div class='exercise'><b>Exercise 1.2 [2 pts]:</b> In the cell below:
    
- Write a line of code that prints to the screen the status of `home_page` (the web page's returned object). You should receive a code of 200 if the request was successful; then,

- Write code that prints the entire contents of `home_page`</div>


In [5]:
# YOUR CODE HERE
print(home_page.status_code)
print(home_page.text)
# END OF YOUR CODE HERE

200
<!doctype html><html lang="en"><head><script type="text/javascript">window.NREUM||(NREUM={});NREUM.info = {"agent":"","beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"NRJS-c11b817f31177e0b4d1","applicationID":"645680062","applicationTime":764.036683,"transactionName":"ZwZaNUEFVhZZAkNRWl5Mdg5BCVkJURtSXGBCChdL","queueTime":0,"ttGuid":"bb5b85950071ccd4","agentToken":null}; (window.NREUM||(NREUM={})).init={privacy:{cookies_enabled:false}};(window.NREUM||(NREUM={})).loader_config={xpid:"VgMCVV5VCBAGUFRUDwEHUFY=",licenseKey:"NRJS-c11b817f31177e0b4d1",applicationID:"645680062"};window.NREUM||(NREUM={}),__nr_require=function(t,e,n){function r(n){if(!e[n]){var i=e[n]={exports:{}};t[n][0].call(i.exports,function(e){var i=t[n][1][e];return r(i||e)},i,i.exports)}return e[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var i=0;i<n.length;i++)r(n[i]);return r}({1:[function(t,e,n){function r(t){try{c.console&&console.log(t)}catch(e){}}var i,o=t("ee"),

<div class='exercise'><b> Exercise 1.3 [1 pt]:</b>
    
In the cell below, create a new BeautifulSoup object that parses the `home_page` as an HTML document (can be done with 1 line of code)</div>

In [6]:
# YOUR CODE HERE
home_page_soup = BeautifulSoup(home_page.text, 'html.parser')
# END OF YOUR CODE HERE

<div class='exercise'><b> Exercise 1.4 [8 pts]:</b>
    
In the cell below, write code that uses the BeautifulSoup object to parse through the home page in order to extract the link for every state. Feel free to use Regular Expressions, in conjunction with any BeautifulSoup parsing. Specifically, the goal is to populate the `state_urls` dictionary by setting each key to be the state name and the value to be the full URL. When complete, there will be 51 keys (50 states + 1 for DC).

### AS A CRITICAL EXAMPLE:###
Within `state_urls`, one of your <key, value> pairs should be:

``"District of Columbia" : "https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/state/district-of-columbia"``

The casing here is **incredibly** important because later, in Exercise 4, you will merge your data with another dataset that has casing of this form. Thus, our key here should be `District of Columbia` and not `District Of Columbia` or `district-of-columbia`.


**NOTES:**
- There are _many_ solutions, but you may find it easiest to use Regular Expression(s)
- Pay attention to the casing example above, so that your later Exercises go smoothly. 
</div>

In [7]:
state_urls = {}

# YOUR CODE HERE
for link in home_page_soup.find_all("a", "jss45"):
    state_urls[link.text] = "https://usafacts.org" + link['href']
# END OF YOUR CODE HERE

Run the cell below to help ensure your formatting is correct and has 51 <key, value> pairs.

In [8]:
# SANITY CHECK
if len(state_urls.keys()) != 51 or \
state_urls["District of Columbia"] != "https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/state/district-of-columbia":
    print("** 1.4 is incorrect")
else:
    print("** 1.4 might be correct")

** 1.4 might be correct


We wish to use the data without having to re-download it every time. So, let's save each webpage to our local hard drive. **NOTE: It's probably okay to download all of the state web pages a few times a day, but it's safer to keep it to a minimum.**

<div class='exercise'><b> Exercise 1.5 [5 pts]:</b>
    
In the cell below, we will iterate through all <key, value> items in `state_urls`. Your job is to make a web request for each URL and save the **contents** out to a file on your hard drive (use `state_dir` as the prefix to the path.) 

**NOTES:**
- Leave the 2 second pause
- You should be saving to a file the actual content of the webpage, not a BeautifulSoup object. That is, you should be able to open the saved files in an editor and see the HTML code, just as you could if you were to view the webpage in your browser and click 'View Page Source'.
- See [official Python documentation](https://docs.python.org/3/tutorial/inputoutput.html#reading-and-writing-files) for details on how to read/write files to disk
- You should have saved 51 different files to your hard drive.
</div>

In [9]:
# 1.5 (4 pts) -- save each webpage to disk
for state, url in state_urls.items():
    
    # YOUR CODE HERE
    with open(state_dir + state, 'w') as f:
        f.write(str(BeautifulSoup(requests.get(url).text, 'html.parser')))
        f.close()
    # END OF YOUR CODE HERE
    
    sleep(2) # LEAVE THIS IN

## 2. Loading and Exploring Data (22 pts)
Now, let's actually use the data! Fortunately, it's saved to our local machine, so we don't need to re-crawl the data every time we wish to access it. We want you to understand that PANDAS is a library of useful data structures and operations, but we also wish to remind you that it isn't magic and it isn't the _only_ way to do Data Science; it's just a tool to help, and you could do the same operations without PANDAS. Thus, here we ask you to perform a few operations without using PANDAS, and then in Exercise 3 we will use PANDAS.

**Terminology Notice:** In the United States, every state is comprised of many **counties.** You can think of a **county** as being a pretty large district. 

First, run the cell below to construct `state_info`

In [10]:
state_info = [(state, state_dir + state) for state in state_urls.keys()]

<div class='exercise'><b> Exercise 2.1 [10 pts]: Parsing and storing data</b>
    
Complete the `load_covid_data()` function, which:

- Takes as input `state_info`, which is a list of tuples: (state name, path to the corresponding file)
- Parses the contents of the file and extracts for **each county**:
    - \# of confirmed cases (total)
    - \# of deaths
    - \# of confirmed cases (per 100k)
- Stores in a **non-PANDAS** data structure named `covid_data` the above 3 pieces of data, **for every county across every state**
- Returns `county_counts`
    <font color='blue'>

**NOTES:**
- To be clear, as of September 7, 2020, the webpage for Alabama currently lists 67 counties. District of Columbia has 1 county, and Wyoming has 23. Here we are asking you to store in `covid_data` *all counties* across every state. So, later, if we were wished to access just Wyoming's information, you could easily retrieve such for each of its 23 counties, or the info for any of the 67 counties in Alabama. 
- `covid_data` **must not be a PANDAS data structure;** it must use a combination of lists and/or dictionaries. It's up to you to decide how to organize this, e.g., a lists of lists of lists, or a list of dictionaries, or a dictionary of dictionaries, or a dictionary of lists of lists, etc. A guiding decision should be ease of access for computing basic stats (Exercises 2.2, 2.3, and 2.4)
</div>

In [11]:
# parses all county-level COVID data from the
# passed-in list of (state, filepath) tuples 
def load_covid_data(state_info):

    covid_data = {}
    
    # YOUR CODE HERE
    for state in state_info:
        counter = 0
        with open(state[1]) as f:
            soup = BeautifulSoup(f.read(), 'html.parser')
            county_data = {}
            raw_data = []
            for link in soup.find_all("td", "MuiTableCell-root-52 MuiTableCell-body-54 MuiTableCell-sizeSmall-56"):
                try:
                    raw_data.append(float(link.text))
                except:
                    raw_data.append(float(link.text.replace(',','')))
            for link in soup.find_all("a", "jss41"):
                county_list = []
                for i in range(0,3):
                    county_list.append(raw_data[counter])
                    counter += 1
                county_data[link.text] = county_list
        covid_data[state[0]] = county_data

    # END OF YOUR CODE HERE
    return covid_data

Run the cell below (no changes necessary) to execute your code above

In [12]:
covid_data = load_covid_data(state_info)

<div class='exercise'><b> Exercise 2.2 [4 pts]: Simple analytics</b>
    
Complete the `calculate_county_stats()` function, which calculates:
1. The single county (and the state to which it belongs) that has the **lowest rate** of COVID cases per 100k people
2. The single county (and the state to which it belongs) that has the **highest rate** of COVID cases per 100k people

**NOTES:**
- Place your resulting variables within the blanks of the `print()` statements that we provide
- These values you report should be Floating point numbers (e.g., 3.4), not Integers (e.g., 3).
- If there are ties, return any of the tied counties
</div>

In [13]:
import sys

def calculate_county_stats(covid_data):
    
    # YOUR CODE HERE
    cases_min = ["","",sys.float_info.max]
    cases_max = ["","",sys.float_info.min]
    for state in covid_data:
        covid_data[state]
        for county in covid_data[state]:
            stat = covid_data[state][county][2]
            if stat < cases_min[2]:
                cases_min = [state, county, stat]
            elif stat > cases_max[2]:
                cases_max = [state, county, stat]
    # END OF YOUR CODE HERE
    print(cases_min[1] + " has the lowest rate of confirmed COVID cases: " + str(cases_min[2]) + " per 100k")
    print(cases_max[1] + " has the highest rate of confirmed COVID cases: " + str(cases_max[2]) + " per 100k")

Run the cell below (no changes necessary) to execute your code above

In [14]:
calculate_county_stats(covid_data)

Bristol Bay Borough has the lowest rate of confirmed COVID cases: 0.0 per 100k
Trousdale County has the highest rate of confirmed COVID cases: 14542.7 per 100k


<div class='exercise'><b> Exercise 2.3 [4 pts]: Simple analytics</b>
    
Complete the `calculate_state_deaths()` function, which calculates:
1. The state that has the **lowest number** of deaths
2. The state that has the **highest number** of deaths

**NOTES:**
- Place your resulting variables within the blanks of the `print()` statements that we provide (don't just manually type your textual answers in the blanks)
- These values you report should be Integers, not Floating point numbers.
- If there are ties, return any of the tied counties

</div>

In [15]:
def calculate_state_deaths(covid_data):
    
    # YOUR CODE HERE
    cases_min = ["",sys.float_info.max]
    cases_max = ["",sys.float_info.min]
    for state in covid_data:
        covid_data[state]
        stat = 0
        for county in covid_data[state]:
            stat += covid_data[state][county][1]
        if stat < cases_min[1]:
              cases_min = [state, stat]
        elif stat > cases_max[1]:
            cases_max = [state, stat]
    # END OF YOUR CODE HERE

    print(cases_min[0] + " has the fewest COVID deaths: " + str(int(cases_min[1])))
    print(cases_max[0] + " has the most COVID deaths: " + str(int(cases_max[1])))

Run the cell below (no changes necessary) to execute your code above

In [16]:
calculate_state_deaths(covid_data)

Alaska has the fewest COVID deaths: 44
New York has the most COVID deaths: 32623


<div class='exercise'><b> Exercise 2.4 [4 pts]: Simple analytics</b>
    
Complete the `calculate_state_deathrate()` function, which calculates:
1. The state that has the **lowest rate** of deaths based on its entire population
2. The state that has the **highest rate** of deaths based on its entire population

**NOTES:**
- To calculate a state's population, we are asserting that is sufficient to sum the population over all counties, and that each county's population can be calculated simply from the data fields stored within `covid_data`.
- **If a county has reported 0 COVID cases,** then we should ignore this county as we estimate its county population. Thus, that county would contribute 0 to its state population total.
- Round your results to the a single person (e.g., "1 out of every 2703 people has died" not 2703.4)
- Place your resulting variables within the blanks of the `print()` statements that we provide (don't just manually type your textual answers in the blanks)
</div>

In [17]:
def calculate_state_deathrate(covid_data):
    
    # YOUR CODE HERE
    death_rate_max = ["",sys.float_info.max]
    death_rate_min = ["",sys.float_info.min]
    for state in covid_data:
        covid_data[state]
        deaths = 0
        pop = 0
        death_rate = 0
        for county in covid_data[state]:
            deaths += covid_data[state][county][1]
            try:
                pop += (covid_data[state][county][0]*100000)/covid_data[state][county][2]
            except:
                pop += 0
        death_rate = pop/deaths
        if death_rate < death_rate_max[1]:
            death_rate_max = [state, death_rate]
        elif death_rate > death_rate_min[1]:
            death_rate_min = [state, death_rate]

    # END OF YOUR CODE HERE
    print(death_rate_min[0] + " has the lowest COVID death rate; 1 out of every " + str(int(round(death_rate_min[1]))) + " people has died")
    print(death_rate_max[0] + " has the highest COVID death rate; 1 out of every " + str(int(round(death_rate_max[1]))) + " people has died")

Run the cell below (no changes necessary) to execute your code above

In [18]:
calculate_state_deathrate(covid_data)

Alaska has the lowest COVID death rate; 1 out of every 16567 people has died
New Jersey has the highest COVID death rate; 1 out of every 554 people has died


## 3. PANDAS (36 pts)
What if we wanted to observe more than just the single-most extreme counties and states? What if we wanted to inspect all states, after having sorted the data by some feature? As you saw in the above exercises, doing the most basic analytics is possible, but it can quickly become cumbersome. As we learned in class, PANDAS is a great library that provides data structures that are highly useful for data analysis.

<div class='exercise'><b> Exercise 3.1 [10 pts]: Converting to PANDAS</b>

In Exercise 2, we worked with `covid_data`, which is comprises of some combination of lists and/or dictionaries.

Complete the `convert_to_pandas()` function, which converts `covid_data` to a PANDAS DataFrame, whereby:
- Each row corresponds to a unique county
- The 5 columns are:
    - county
    - state
    - \# total covid cases (Integer)
    - \# covid cases per 100k (Integer)
    - \# covid deaths (Integer)
- The columns should be titled **exactly** as listed above

**NOTE:**
- If there exists multiple counties with the same name, each of which belonging to a different state, then there should be a distinct row for each.
- The 3 columns that correspond to COVID counts should all be Integers (e.g., 1498), not Floating point digits (e.g., 1498.0)
</div>

In [19]:
def convert_to_pandas(covid_data):
    
    # YOUR CODE HERE
    covid_df = pd.DataFrame()
    for state in covid_data:
        for county in covid_data[state]:
            l = covid_data[state][county]
            row = pd.Series([county, state, l[0], l[2], l[1]])
            covid_df = covid_df.append(row,ignore_index=True)
    covid_df = covid_df.rename(columns={0:'county',1:'state',2:'total covid cases',3:'covid cases per 100k',4:'covid deaths'})
    # END OF YOUR CODE HERE
    return covid_df

Run the cell below (no changes necessary) to execute your code above

In [20]:
covid_df = convert_to_pandas(covid_data)
covid_df

,county,state,total covid cases,covid cases per 100k,covid deaths
0,Autauga County,Alabama,1585.0,2837.0,24.0
1,Baldwin County,Alabama,4978.0,2229.9,47.0
2,Barbour County,Alabama,801.0,3244.8,7.0
3,Bibb County,Alabama,610.0,2723.9,9.0
4,Blount County,Alabama,1464.0,2531.7,13.0
...,...,...,...,...,...
3137,Sweetwater County,Wyoming,317.0,748.6,2.0
3138,Teton County,Wyoming,476.0,2028.6,1.0
3139,Uinta County,Wyoming,312.0,1542.6,2.0
3140,Washakie County,Wyoming,111.0,1422.2,6.0


<div class='exercise'><b> Exercise 3.2 [5 pts]: Simple analytics</b>

Complete the `calculate_county_stats2()` function, **which should obtain identical information as problem 2.2, but now using the PANDAS `covid_df` DataFrame.**

That is, it should calculates:
1. the single county (and the state to which it belongs) that has the **lowest rate** of COVID cases per 100k people
2. the single county (and the state to which it belongs) that has the **highest rate** of COVID cases per 100k people

**NOTES:**
- If there are ties, return any of the tied counties
- Place your resulting variables within the `print()` statements that we provide (don't just manually type your textual answers in the blanks)
- The values you report should be Floating point numbers (e.g., 3.4), not Integers (e.g., 3).

</div>

In [21]:
def calculate_county_stats2(covid_df):

    # YOUR CODE HERE
    min_row = covid_df['covid cases per 100k'].idxmin()
    max_row = covid_df['covid cases per 100k'].idxmax()
    # END OF YOUR CODE HERE
    print(covid_df.loc[min_row,'county'] + " (" + covid_df.loc[min_row,'state'] + ") has the lowest rate of confirmed COVID cases: " + str(covid_df.loc[min_row,'covid cases per 100k']) + " per 100k")
    print(covid_df.loc[max_row,'county'] + " (" + covid_df.loc[max_row,'state'] + ") has the highest rate of confirmed COVID cases: " + str(covid_df.loc[max_row,'covid cases per 100k']) + " per 100k")

Run the cell below (no changes necessary) to execute your code above

In [22]:
calculate_county_stats2(covid_df)

Bristol Bay Borough (Alaska) has the lowest rate of confirmed COVID cases: 0.0 per 100k
Trousdale County (Tennessee) has the highest rate of confirmed COVID cases: 14542.7 per 100k


<div class='exercise'><b> Exercise 3.3 [5 pts]: Simple analytics</b>
    
Complete the `calculate_state_deaths2()` function, **which should obtain identical information as problem 2.3, but now using the PANDAS `covid_df` DataFrame.**
1. the state that has the **lowest number** of deaths
2. the state that has the **highest number** of deaths

**NOTES:**
- If there are ties, return any of the tied counties
- Place your resulting variables within the `print()` statements that we provide (don't just manually type your textual answers in the blanks)
- The values you report should be Integers, not Floating point numbers.
</div>

In [23]:
def calculate_state_deaths2(covid_df):
    
    # YOUR CODE HERE
    df = covid_df.groupby(by='state').sum()['covid deaths']
    # END OF YOUR CODE HERE
    print(df.idxmin() + " has the fewest COVID deaths: " + str(int(df.min())))
    print(df.idxmax() + " has the most COVID deaths: " + str(int(df.max())))

Run the cell below (no changes necessary) to execute your code above

In [24]:
calculate_state_deaths2(covid_df)

Alaska has the fewest COVID deaths: 44
New York has the most COVID deaths: 32623


<div class='exercise'><b> Exercise 3.4 [5 pts]: Simple analytics</b>
    
Complete the `calculate_state_deathrate2()` function, **which should obtain identical information as problem 2.4, but now using the PANDAS `covid_df` DataFrame.**

1. The state that has the **lowest rate** of deaths based on its entire population
2. The state that has the **highest rate** of deaths based on its entire population

**NOTES:**
- Just as in, 2.4, to calculate a state's population, we are asserting that is sufficient to sum the population over all counties -- and that each county's population can be calculated simply from the data fields stored within `covid_data`.
- Just as in 2.4, counties with 0 COVID cases should contibute 0 to the total population of the state.
- Round your results to the a single person (e.g., "1 out of every 2703 people has died" not 2703.4)
- Place your resulting variables within the blanks of the `print()` statements that we provide (don't just manually type your textual answers in the blanks)
</div>

In [25]:
def calculate_state_deathrate2(covid_df):
    
    # YOUR CODE HERE

    pop = ((covid_df['total covid cases'])*100000)/covid_df['covid cases per 100k']
    
    covid_df['population'] = pop
 
    dr = (covid_df.groupby(by = 'state')['population'].sum())/(covid_df.groupby(by= 'state')['covid deaths'].sum())
    
    # END OF YOUR CODE HERE
    print(dr.idxmax() + " has the lowest COVID death rate; 1 out of every " + str(int(dr.max()))+ " people has died")
    print(dr.idxmin() + " has the highest COVID death rate; 1 out of every " + str(int(dr.min())) + " people has died")

Run the cell below (no changes necessary) to execute your code above

In [26]:
calculate_state_deathrate2(covid_df)

Alaska has the lowest COVID death rate; 1 out of every 16566 people has died
New Jersey has the highest COVID death rate; 1 out of every 553 people has died


These are highly alarming and tragic statistics, and doing calculations like this can really put the severity of the virus into a grounded perspective. In order to perfectly understand the virus and its spread, everyone would be tested and we would have contact tracing. Without getting into socio-political issues, our point is that (1) we wish to better understand the virus' effects; (2) naturally, any real-world data is messy, and thus we will never have _perfect_ data.


Let's now attempt to understand _some_ of the uncertainty around our COVID data. It's reasonable to believe that the # of COVID deaths is fairly reliable. That is, there are inevitably some false negatives -- people who died of COVID but were not accounted for, as other conditions were listed as the cause. However, the number of false positives is probably minimal -- if someone was denoted as dying from COVID, it's probably true. It's also the case that every disease has a mortality rate. For example, if 1,000 randomly-selected people contracted COVID, $N\%$ of them will die. We'd imagine that this percentage should be pretty constant throughout all people in the United States. Of course, we can think of reasons for this rate to not be perfectly consistent, as some people are at higher risk (e.g., older folks, people with pre-existing conditions, etc). Yet, we can imagine that this natural *variance* in the population to be fairly uniform throughout the USA at large. To this end, if all counties were equal in their **testing**, we ought to see a consistent ratio between: (a) the # of people who died from COVID; and (b) the # of people who tested positive for COVID. Within the medical domain, this ratio is referred to as the `case_fatality_rate`. For example, if 750 people tested positive for COVID, and 75 of those people died, then our `case_fatality_rate` would be 0.1 (meaning 10%).

<div class='exercise'><b>Exercise 3.5 [5 pts]: Further analytics</b>
    
Complete the `add_death_stats()` function below, which should add 2 new columns:
- `case_fatality_rate` and
- `# covid deaths per 100k`

And return the updated DataFrame **sorted by case_fatality_rate in ascending order** 

**NOTES:**

- `add_death_stats()` should return a new DataFrame that has 8 columns:
    - county
    - state
    - population
    - \# total covid cases
    - \# covid cases per 100k
    - \# total covid deaths
    - \# covid deaths per 100k
    - case_fatality_rate
- DataFrame should be sorted by `case_fatality_rate` in ascending order
- Again, the values for `case_fatality_rate` should be < 1. A value of 1 would mean that 100% of people who tested positive for COVID also died.
- `# covid deaths per 100k` is simply defined as the # of COVID deaths for every 100,000 people. We calculate this on a per-county basis.
</div>

In [27]:
def add_death_stats(covid_df):
    
    # YOUR CODE HERE
    pop_100k = covid_df['population']/100000
    death_100k = covid_df['covid deaths']/pop_100k
    covid_df['covid deaths per 100k'] = death_100k
    
    cfr = covid_df['covid deaths']/covid_df['total covid cases']
    covid_df['case_fatality_rate'] = cfr
    
    covid_df = covid_df.sort_values(by = ['case_fatality_rate'], ascending=True)
    
    # END OF YOUR CODE HERE
    return covid_df
    
    #We found 15 NaN values because there appears to be 15 counties that dont have a single covid case.  Because of this, there's no way for us to estimate the population and the other variables that require population estimates to calculate.
    #We considered substituting the NaN values for zero, but we know the population isn't zero therefore the only substitutable variables that would have made sense would have been deaths per 100k and case fatality rate
    #While we could have subbed 0 for those, we decided to leave the NaN variables as it would preserve the continuity of our analysis and we wanted as little user meddling as possible 
    #print(covid_df.isnull().sum())

Run the cell below (no changes necessary) to execute your code above

In [28]:
covid_updated = add_death_stats(covid_df)
covid_updated

,county,state,total covid cases,covid cases per 100k,covid deaths,population,covid deaths per 100k,case_fatality_rate
3141,Weston County,Wyoming,23.0,332.0,0.0,6927.710843,0.0,0.0
1759,Pershing County,Nevada,21.0,312.3,0.0,6724.303554,0.0,0.0
2793,Rich County,Utah,13.0,523.6,0.0,2482.811306,0.0,0.0
1760,Storey County,Nevada,9.0,218.3,0.0,4122.766835,0.0,0.0
2792,Piute County,Utah,6.0,405.7,0.0,1478.925314,0.0,0.0
...,...,...,...,...,...,...,...,...
1751,Esmeralda County,Nevada,0.0,0.0,0.0,NaN,NaN,NaN
2242,Wheeler County,Oregon,0.0,0.0,0.0,NaN,NaN,NaN
2539,Borden County,Texas,0.0,0.0,0.0,NaN,NaN,NaN
2657,King County,Texas,0.0,0.0,0.0,NaN,NaN,NaN


<div class='exercise'><b> Exercise 3.6 [6 pts]: Reflection:</b> Data Analysis allows us to better understand a system or scenario.
</div>

<div class='exercise'><b>Exercise 3.6: Q1 (2  of 6 pts)</b>
    
Having looked at the results from Exercises 3.3, 3.4, and 3.5, what are some trends you've noticed and any conclusions you have? (2-3 sentences)?</div>

YOUR RESPONSE HERE

*Answer:* Consistent across our results from Exercises 3.3, 3.4, and 3.5 is that Alaska has the lowest COVID numbers in total deaths and death rates, and there are multiple counties in Alaska with few to no COVID cases at all (this isn't surprising given how remote Alaska is and the lack of dense population across most of the state). One surprising result we found was that Trousdale County in Tennessee had the highest rate of confirmed cases, but from further research we discovered that Trousdale County (and nearby Bledsoe County) have the highest rate of COVID-19 because of outbreaks among prison populations who lack the ability to social distance. We found that New York had the most COVID deaths and that New Jersey had the highest rate of COVID cases, which is unsurprising given that many people cross back and forth from New York to New Jersey, particularly around the Manhattan area which is likely the source of most of the cases that we see.

<div class='exercise'><b>Exercise 3.6: Q2 (2 of 6 pts)</b>
    
Having looked at the results from Exercise 3.5 (i.e., `covid_updated` DataFrame), do you think the original data is reliable and accurate? Are there any potential biases that you're aware of or concerned about? Please explain (3-5 sentences).</div>

YOUR RESPONSE HERE

*Answer:* The original data obtained from usafacts.org is reliable and accurate with likely no biases. Reading through the detailed methodology that usafacts.org provides, they are taking the official numbers reported by public health agencies in each state or county (there are different reporting rules for each state) and verifying those numbers with multiple sources, leading us to believe that these are unbiased observations of the (theoretical) true number of covid cases. We are concerned about potential estimation biases in our calculated columns in the covid_updated Dataframe due to lack of precision in the numbers that we use to estimate population (and subsequently covid deaths per 100k) that bias our estimate away from the (theoretical) true population of each county. Moreover, our estimation method allows for fractional values of population, introducing a source of bias since we know that the true population number for each county must be integral (that is we cannot have fractional people). For the purposes of precision, we've decided to leave population values as floats instead of truncating them all to integers. Since covid_fatality_rate is directly calculated from the reported covid data, we are not estimating this rate so there is no bias.

<div class='exercise'><b>Exercise 3.6: Q3 (1 of 6 pts)</b>
    
If a county has 15 confirmed deaths, how many cases would you expect? What would you expect its population to be? Explain why (1-2 sentences in total)?

**NOTE:** For this question, we aren't evaluating the accuracy of your answer but your thought-process and reasoning.
</div>

YOUR RESPONSE HERE

In [29]:
predicted_total_cases=15/covid_updated['case_fatality_rate'].mean()
print("Predicted Total COVID-19 Cases: "+ str(predicted_total_cases))
predicted_population = (predicted_total_cases*100000)/covid_updated['covid cases per 100k'].mean()
print("Predicted Population: "+ str(predicted_population))

Predicted Total COVID-19 Cases: 689.4044294018875
Predicted Population: 42217.093786272024


*Answer:* We are predicting using simple averages of the case fatality rate and covid cases per 100k across our entire dataset. A potentially interesting avenue would be to find the variance of case fatality rate and covid cases per 100k such that we may find an empirical distribution of our dataset and could then use that to give a 95% prediction interval.

<div class='exercise'><b>Exercise 3.6: Q4 (1 of 6 pts)</b>
    
What further questions do you wish to answer about COVID, including ones that may not be possible to answer from this data alone (e.g., Is there a correlation between the average age of people in a county and the # of COVID deaths)? Write at least 3 of your questions.</div>

YOUR RESPONSE HERE

*Answer:* 
1. Much research has come out regarding the asymmetric COVID death rates between whites or non-whites: Is the difference due to associated confounding factors (including income inequality, employment rates, and geography)?
2. Does hospital bed availability make a difference in the COVID fatality rates? Namely, does a shortage of hospital beds in densely populated locations, like Manhattan versus rural Alaska, influence the COVID fatality rates?
3. Does the political party of a state's governor produce a significant effect on the likelihood of an individual in that state catching COVID and surviving it? On a more detailed scale, does the political affiliation of a county's political leadership affect the likelihood that an individual may catch COVID and the likelihood an individual will survive COVID?

## 4. MORE DATA (25 pts)
In order to better understand how COVID (and the testing thereof) has impacted our world, we could look at how it relates to demographics, income, education, health, and political voting. For this exercise, we will make use of `election2016_by_county.csv`.

<div class='exercise'><b>Exercise 4.1 [4 pts]: Load more data</b>

Complete the `merge_data()` function, which should:
1. First, load `election2016_by_county.csv` as a new DataFrame.
2. Then, using the state and county names (case-sensitive) in both DataFrames, merge this new DataFrame with your existing `covid_updated`.
3. Return the merged DataFrame

The returned `merged` DataFrame should contain all 7 columns from `covid_updated`:
- county
- state
- \# total covid cases
- \# covid cases per 100k
- \# covid deaths
- population
- case_fatality_rate

along with these 14 columns from `election2016_by_county.csv`:
- hispanic
- minority
- female
- unemployed
- income
- nodegree
- bachelor
- inactivity
- obesity
- density
- cancer
- votergap
- trump
- clinton

**NOTES:**
- We are dropping two columns from `election2016_by_county.csv`:
    - fipscode
    - population
- Do not attempt to manually fix any of the state or county names. That is, **our merging should require the state and county names to be identical (case-sensitive) between the two DataFrames.** If there is a discrepancy between the two, do not worry about adjusting these names to find a perfect match.

**HINT:** there are many ways to solve this, but you may find the [pandas.merge()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html) function can be really helpful
</div>

In [30]:
def merge_data(covid_updated, filepath):
    
    # YOUR CODE HERE
    df = pd.read_csv(filepath).drop(columns=['fipscode','population'])
    # END OF YOUR CODE HERE
    return covid_updated.merge(df, left_on=['county','state'],right_on=['county','state'])

Run the cell below (no changes necessary) to execute your code above

In [31]:
merged = merge_data(covid_updated, 'election2016_by_county.csv')

As mentioned above, the merging requires exact matching between the two DataFrames' `state` and `county` columns. Thus, some mismatches will occur, yielding our `merged` DataFrame to have fewer rows than `covid_updated` and `election2016_by_county.csv`.

<div class='exercise'><b>Exercise 4.2 [5 pts]: Data Construction / Understanding</b>
</div>

<div class='exercise'><b>Exercise 4.2: Q1 (1 of 5 pts)</b>
    
Compared to `covid_updated`, how many rows were lost during this merging process to create `merged`? Running the cell below should print to the screen your answer.
</div>

In [32]:
# YOUR CODE HERE
print(len(covid_updated.index)-len(merged.index))
# END OF YOUR CODE HERE

63


<div class='exercise'><b>Exercise 4.2: Q2 (2 of 5 pts)</b>  

List the county and state of *at least 3* such rows that exist in `covid_updated` but didn't make it into `merged`. Running the cell below should print to the screen your answer.
</div>

In [33]:
# YOUR CODE HERE
pd.concat([covid_updated[['county','state']],merged[['county','state']]]).drop_duplicates(keep=False)
# YOUR CODE HERE

,county,state
2921,"Covington, City of",Virginia
92,Valdez–Cordova Census Area,Alaska
93,"Wrangell, City and Borough of",Alaska
91,Southeast Fairbanks Census Area,Alaska
89,"Sitka, City and Borough of",Alaska
...,...,...
2925,"Falls Church, City of",Virginia
87,Petersburg Borough,Alaska
90,"Skagway, Municipality of",Alaska
94,"Yakutat, City and Borough of",Alaska


<div class='exercise'><b>Exercise 4.2: Q3 (2 of 5 pts)</b>  
? If we needed to be highly thorough and needed comprehensive data coverage, do you have any suggestions on how we could quickly, soundly fix most or all of them? (Write 2-3 sentences.)
    
<b>NOTE: Please do not actually fix these mismatches; for this Exercise, it's okay that the `merged` DataFrame is smaller than `covid_updated`</b>
</div>

*Answer:* From inspection of the election csv data, we discovered that the 63 rows that were dropped were not matched because of syntatical differences between the names of the county in our original COVID dataset and this new dataset. We could use Regular Expressions to allow for flexibility in what counts as a match for the county names. Differences in spaces, commas, or words like "Census" "Area" "Borough" "City" "of" "the" "county" etc. aren't meaningful and led to these 63 rows being unmatched. For example, the election csv has "Covington city" in Virginia while our COVID dataset has "Covington, City of" in Virgina. The syntatical differences aren't meaningful and we could use Regular Expressions so that we match only on "Covington" and "Virginia."

This past example demonstrates how easy it is for data to become messy. It also shows the importance of paying close attention to your data in order to understand what you are working with.

Our `case_fatality_rate` column can be viewed as an approximation of how effective and thorough *COVID testing* is for a given county.

Our `# covid deaths` column can be viewed as an extreme indication of how severe *COVID* has impacted a given county.

Our `# covid cases per 100k` column be viewed as middle-ground between the two aforementioned features. That is, it measures the impact of the disease and is influenced by the thoroughness of COVID testing.

Using these three informative features, we can inspect how impacted each county is, while correlating this with other features of each county, such as income-level, health metrics, demographics, etc. 

<div class='exercise'><b>Exercise 4.3 [2 pts]: Cleaning the data</b>

Before we do any further analysis, we first notice that some counties haven't encountered a single COVID death (usually ones with very small populations), thus providing us with little information. Write code in the cell below to update the `merged` DataFrame so that all rows with 0 deaths are removed.

In [34]:
# YOUR CODE HERE
merged = merged[merged['covid deaths']>0]
# END OF YOUR CODE HERE

Running `.describe()` allows us to quickly see summary statistics of our DataFrame

In [35]:
merged.describe()

,total covid cases,covid cases per 100k,covid deaths,population,covid deaths per 100k,case_fatality_rate,hispanic,minority,female,unemployed,income,nodegree,bachelor,inactivity,obesity,density,cancer,votergap,trump,clinton
count,2519.000000,2519.000000,2519.000000,2.519000e+03,2519.000000,2519.000000,2519.000000,2519.000000,2519.000000,2519.000000,2519.000000,2519.000000,2519.000000,2519.000000,2519.00000,2519.000000,2496.000000,2515.000000,2515.000000,2515.000000
mean,2512.944819,1840.432989,75.754268,1.256324e+05,46.277522,0.026504,9.807996,16.264299,50.073896,5.658039,47104.389440,15.564907,20.150179,26.133664,31.19206,262.259468,227.034696,29.253846,62.381442,33.127597
std,9478.623844,1403.378264,353.938762,3.678286e+05,50.498100,0.023223,14.309347,16.264531,2.125094,1.887679,12320.793686,6.832968,9.038016,5.299880,4.56083,1858.520644,54.567489,30.121353,15.260465,14.983753
min,6.000000,56.700000,1.000000,4.039938e+02,0.910759,0.001088,0.426000,1.253000,28.479000,1.800000,21658.000000,1.900000,6.000000,8.100000,11.80000,0.100000,46.200000,-88.725000,4.122000,6.883000
25%,203.500000,887.150000,3.000000,1.633776e+04,13.129515,0.011780,2.247000,4.777500,49.656500,4.400000,38487.500000,10.400000,13.800000,22.700000,28.50000,22.000000,192.875000,12.715500,53.918000,21.806500
50%,526.000000,1509.100000,10.000000,3.490926e+04,29.005072,0.020048,4.237000,9.562000,50.489000,5.400000,45065.000000,14.300000,17.700000,26.000000,31.30000,51.600000,228.850000,35.645000,65.185000,29.775000
75%,1473.500000,2431.500000,34.000000,9.095212e+04,59.438696,0.033898,10.083000,22.698000,51.121000,6.600000,52616.000000,19.950000,24.300000,29.700000,34.10000,130.200000,262.500000,51.748500,73.691000,41.582000
max,254656.000000,14542.700000,7306.000000,1.003927e+07,484.806069,0.333333,95.824000,93.411000,56.739000,24.000000,122641.000000,53.300000,72.000000,41.400000,47.60000,69468.400000,458.300000,83.972000,90.855000,92.847000


Using the information reported from `.describe()`, we can imagine dividing our DataFrame into 4 separate bins, based on the distribution for any given feature. Specifically, based on a particular feature:
- the $1^{st}$ bin will be the data that has values between the **min** and **25%**
- the $2^{nd}$ bin will be the data that has values between **25%** and **50%**
- the $3^{rd}$ bin will be the data that has values between **50%** and **75%**
- the $4^{th}$ bin will be the data that has values between **75%** and **max**

<div class='exercise'><b>Exercise 4.4 [3 pts]: Partitioning our data</b>
    
Complete the `partition_df()` function, which takes as input:
- DataFrame to work with
- feature (e.g., obesity) to filter by
- minimum value
- maximum value

and outputs:
- a subset of the DataFrame that has values between the passed-in minimum and maximum values (inclusively) for the passed-in feature.

For example, if we called `partition_df(merged, 'obesity', 30, 45)`, it should return a subset of the `merged` DataFrame that has obesity values between 30 and 45 (and including the boundary values of 30 and 45).
</div>

In [36]:
def partition_df(df, column_name, min, max):
    # YOUR CODE HERE
    return df[(df[column_name]>=min) & (df[column_name]<=max)]
    # END OF YOUR CODE HERE

<div class='exercise'><b>Exercise 4.5: [4 pts] Exploratory Data Analysis</b>
    
Identify a few features that you're interested in, and inspect if there's any correlation with the COVID data. Specifically, simply run your `partition_df()` function below, many times, each with a different subset of the data -- select a range of values and a particular feature. For example, if I'm interested in __cancer__, I could look at the 4 quartiles (per `.describe()`) and use those ranges of values as I repeatedly execute `partition_df()`. For this exercise, after running the function several times, **write 3-5 sentences about any patterns or correlations you noticed or didn't notice but expected to find.**
</div>

In [54]:
# YOUR CODE HERE
# END OF YOUR CODE HERE

#0-25 quartile minority population percentage
partition_df(merged, 'minority', 0.426000, 4.777500).describe()

,total covid cases,covid cases per 100k,covid deaths,population,covid deaths per 100k,case_fatality_rate,hispanic,minority,female,unemployed,income,nodegree,bachelor,inactivity,obesity,density,cancer,votergap,trump,clinton
count,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,628.000000,630.000000,630.000000,630.000000
mean,417.584127,1152.998095,11.430159,32279.492459,30.225135,0.028643,5.828783,3.248851,50.179798,5.390159,46279.212698,14.259206,17.494603,26.730159,31.530635,184.566667,239.212102,43.346135,69.284124,25.937994
std,1576.425252,825.755518,64.529290,48438.955197,37.018306,0.029361,14.462325,0.849635,0.947375,1.876327,9462.134584,7.053257,6.405833,4.795803,3.716478,1438.636933,52.313415,23.645999,12.120672,11.656287
min,6.000000,56.700000,1.000000,1501.125844,1.261888,0.001088,0.426000,1.253000,41.607000,1.800000,23047.000000,1.900000,7.000000,8.900000,14.300000,0.700000,68.300000,-60.115000,18.972000,6.883000
25%,98.000000,595.350000,1.000000,12397.625460,8.396263,0.010467,1.448500,2.614500,49.705250,4.000000,39739.250000,9.300000,13.200000,23.700000,29.400000,21.575000,206.725000,34.921000,64.571000,18.651750
50%,194.500000,958.750000,3.000000,22080.833984,17.953353,0.020050,2.128000,3.236500,50.317500,5.100000,45764.000000,12.500000,16.400000,26.500000,31.700000,48.850000,239.350000,47.977500,71.628500,23.677000
75%,393.000000,1474.550000,9.000000,38817.175358,36.901524,0.036834,3.855750,3.970750,50.764750,6.400000,52032.250000,17.475000,20.300000,29.500000,33.875000,99.125000,271.850000,58.224750,76.997250,29.840000
max,29446.000000,6019.900000,1344.000000,868716.072693,311.315625,0.333333,95.824000,4.777000,52.973000,13.600000,92560.000000,53.300000,56.400000,41.400000,40.700000,35369.200000,394.000000,83.972000,90.855000,79.086000


In [53]:
#25-50 quartile minority population percentage
partition_df(merged, 'minority', 4.777500, 9.562000).describe()

,total covid cases,covid cases per 100k,covid deaths,population,covid deaths per 100k,case_fatality_rate,hispanic,minority,female,unemployed,income,nodegree,bachelor,inactivity,obesity,density,cancer,votergap,trump,clinton
count,629.000000,629.000000,629.000000,629.000000,629.000000,629.000000,629.000000,629.000000,629.000000,629.000000,629.000000,629.000000,629.000000,629.000000,629.000000,629.000000,616.000000,629.000000,629.000000,629.000000
mean,916.332273,1483.038951,21.254372,64869.391174,31.658270,0.024013,13.205693,6.867439,49.758598,5.137838,48470.667727,14.687440,20.772178,24.658983,29.826868,199.530843,223.442208,36.505302,65.499583,28.994264
std,1689.504400,1094.182330,58.065185,84553.204852,33.306326,0.021926,17.416822,1.389999,1.861688,1.709574,10810.686203,7.047244,8.409415,5.189308,4.296802,807.242964,57.857734,24.353295,12.825956,11.724524
min,7.000000,142.400000,1.000000,403.993767,1.112500,0.001112,0.655000,4.778000,29.748000,2.200000,24868.000000,2.600000,6.900000,8.100000,11.800000,0.300000,46.200000,-50.810000,20.174000,7.382000
25%,185.000000,718.600000,2.000000,16843.865727,10.198851,0.009804,2.725000,5.635000,49.354000,4.000000,41139.000000,9.500000,14.700000,21.700000,27.500000,15.500000,187.750000,21.682000,58.082000,20.368000
50%,450.000000,1215.400000,6.000000,37266.745766,20.173833,0.017118,5.276000,6.675000,50.202000,5.000000,46513.000000,13.100000,18.700000,24.500000,30.200000,43.800000,225.950000,40.183000,67.134000,27.200000
75%,971.000000,1869.800000,18.000000,79594.594595,40.906094,0.030481,15.308000,8.104000,50.756000,6.000000,53783.000000,18.400000,25.600000,27.800000,32.800000,106.100000,263.650000,54.051000,74.299000,36.416000
max,21731.000000,8858.600000,1037.000000,839229.165058,247.528571,0.153846,82.810000,9.517000,55.756000,21.800000,107250.000000,46.800000,58.200000,41.300000,39.300000,13731.400000,404.700000,81.693000,89.947000,70.984000


In [52]:
#50-75 quartile minority population percentage
partition_df(merged, 'minority', 9.562000, 22.698000).describe()

,total covid cases,covid cases per 100k,covid deaths,population,covid deaths per 100k,case_fatality_rate,hispanic,minority,female,unemployed,income,nodegree,bachelor,inactivity,obesity,density,cancer,votergap,trump,clinton
count,630.000000,630.000000,630.000000,6.300000e+02,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,627.000000,629.000000,629.000000,629.000000
mean,3424.639683,1922.198413,92.657143,1.755661e+05,42.339691,0.024717,12.920489,14.851571,49.920263,5.399683,50680.106349,14.830952,22.643016,24.950952,30.026984,434.104762,224.472089,29.479159,62.395405,32.916237
std,9903.096903,1456.230959,264.139976,3.192095e+05,36.695767,0.020766,14.087351,3.735768,2.284988,1.667562,13010.625649,6.250640,9.687163,5.221722,4.111940,3268.470175,55.867504,27.648846,14.462937,13.259865
min,10.000000,125.900000,1.000000,1.398015e+03,0.910759,0.001452,1.085000,9.562000,28.479000,1.900000,27882.000000,3.700000,7.500000,11.600000,16.300000,0.100000,94.700000,-62.763000,16.131000,7.771000
25%,443.250000,1044.200000,7.000000,2.756084e+04,15.607861,0.011336,3.848750,11.552250,49.609250,4.300000,41426.750000,10.200000,15.425000,21.500000,27.500000,22.400000,188.250000,9.863000,52.480000,22.195000
50%,1177.000000,1571.750000,20.500000,7.137274e+04,31.403186,0.018430,7.005500,13.925500,50.537500,5.200000,47340.000000,13.600000,20.450000,24.900000,30.100000,61.100000,223.900000,33.805000,64.142000,30.384000
75%,3038.000000,2375.450000,65.000000,1.924550e+05,55.138920,0.030922,16.427000,17.962500,51.081750,6.200000,56699.750000,18.500000,28.300000,28.200000,32.975000,171.025000,258.850000,51.381000,73.840000,42.013000
max,164299.000000,14542.700000,3154.000000,4.485379e+06,218.340299,0.171340,82.673000,22.697000,54.798000,24.000000,100511.000000,36.400000,55.600000,39.900000,39.700000,69468.400000,398.400000,79.352000,88.849000,78.895000


In [49]:
#75-100 quartile minority population percentage
partition_df(merged, 'minority', 22.698000, 93.411000).describe()

,total covid cases,covid cases per 100k,covid deaths,population,covid deaths per 100k,case_fatality_rate,hispanic,minority,female,unemployed,income,nodegree,bachelor,inactivity,obesity,density,cancer,votergap,trump,clinton
count,630.000000,630.000000,630.000000,6.300000e+02,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,630.000000,625.000000,627.000000,627.000000,627.000000
mean,5290.688889,2802.929206,177.588889,2.297184e+05,80.863785,0.028640,7.282413,40.074417,50.436425,6.703651,42989.739683,18.480635,19.691905,28.192222,33.381587,230.736032,220.910400,7.593512,52.303638,44.710153
std,15516.096651,1533.204341,637.496198,6.359823e+05,68.544910,0.019160,8.106144,14.648548,2.869403,1.881734,14138.067403,6.080345,10.374794,5.207258,5.096825,619.391909,50.192206,31.580338,15.915766,15.723441
min,15.000000,225.800000,1.000000,1.327007e+03,1.488889,0.001189,0.559000,22.699000,31.624000,1.900000,21658.000000,4.800000,6.000000,13.400000,14.700000,0.300000,70.000000,-88.725000,4.122000,15.894000
25%,411.750000,1774.675000,9.000000,1.481200e+04,31.842650,0.015362,2.493750,29.303250,50.099500,5.300000,34008.500000,13.800000,12.500000,25.200000,30.400000,27.850000,191.000000,-14.843500,41.146000,32.522000
50%,959.000000,2579.850000,24.000000,3.297306e+04,61.358464,0.024169,4.324000,35.720000,51.246500,6.400000,38907.500000,18.650000,16.150000,28.600000,33.500000,55.600000,223.100000,13.188000,55.216000,42.256000
75%,3232.750000,3456.575000,75.000000,1.479419e+05,105.616731,0.037146,7.983000,47.723500,51.889250,7.700000,48017.000000,22.700000,23.775000,31.900000,36.675000,153.150000,252.700000,32.539000,64.657000,55.829500
max,254656.000000,14522.800000,7306.000000,1.003927e+07,484.806069,0.110239,55.119000,93.411000,56.739000,16.900000,122641.000000,40.500000,72.000000,40.800000,47.600000,9856.500000,458.300000,65.434000,81.327000,92.847000


*Answer:* We were interested in comparing the quartiles of counties based on their percentage minority population. As we expected (from most of the news coming out about the issue) we find that as we move up the quartiles of minority percentages, those counties with greater percentage minorities have higher total covid cases, covid cases per 100k, covid deaths, and covid deaths per 100k. Surprisingly, we also find that population increases as we move up the quartiles, but perhaps that's a factor of racial inequality forcing minorities to live in more densely populated areas which could also contribute to their higher rates of COVID infections and deaths. We were most curious in obswerving the results for the case fatality rates, because some initial news reports regarding minority populations having higher rates of COVID attempted to make the argument that there could be melanin (related to Vitamin D uptake) or genetic/DNA factors to blame. We find in the data that there is no significant relation between case fatality rates and minority population percentages, showing that case fatality rates may be just inherent to the virus itself. This conclusion is fairly important, because it would dispell many (frankly racialized) arguments that somehow minority populations are inherently more susceptible to COVID when in fact the reason is more likely based on racial and income inequality.

`.describe()` provides these nice summary statistics over any portion of data that we give it. Instead of iteratively inspecting several subsets of the data, let's actually split our DataFrame into new categories; instead of representing all features by floating point numbers, let's create new _categorical_ names for feature(s) based on their numbers. The code below does just this. It creates a new column, `income group` that has 4 possible values, each one corresponding to a quartile of the original `income` values. 

Run the cell below.

In [38]:
bins = [0, 38000, 45000, 52000, 200000]
names = ['income-group-1', 'income-group-2', 'income-group-3', 'income-group-4']
d = dict(enumerate(names, 1))
merged['income group'] = np.vectorize(d.get)(np.digitize(merged['income'], bins))
merged

,county,state,total covid cases,covid cases per 100k,covid deaths,population,covid deaths per 100k,case_fatality_rate,hispanic,minority,...,nodegree,bachelor,inactivity,obesity,density,cancer,votergap,trump,clinton,income group
548,Summit County,Utah,919.0,2180.6,1.0,42144.363937,2.372797,0.001088,11.723,4.673,...,6.7,50.1,11.1,14.4,10.9,165.7,-15.353,35.564,50.916,income-group-4
549,La Crosse County,Wisconsin,1799.0,1524.4,2.0,118013.644713,1.694719,0.001112,1.806,8.258,...,6.0,30.8,19.9,28.3,253.8,182.6,-9.585,42.023,51.608,income-group-3
550,Russell County,Alabama,1682.0,2902.0,2.0,57960.027567,3.450654,0.001189,5.285,47.987,...,18.7,15.5,30.7,36.4,82.6,216.4,-1.918,48.129,50.047,income-group-1
551,Chattahoochee County,Georgia,1584.0,14522.8,2.0,10906.987633,18.336869,0.001263,15.676,28.352,...,7.0,31.5,24.5,27.3,45.3,70.0,11.055,53.902,42.847,income-group-3
552,Effingham County,Illinois,751.0,2208.3,1.0,34008.060499,2.940479,0.001332,1.909,2.037,...,8.7,20.2,21.9,31.8,30.2,278.4,60.489,78.110,17.621,income-group-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3062,Monroe County,Ohio,106.0,776.3,18.0,13654.515007,131.824528,0.169811,0.479,2.158,...,14.2,10.3,35.4,36.8,252.1,220.3,46.874,71.508,24.634,income-group-2
3063,Orleans County,New York,321.0,795.5,55.0,40351.979887,136.300623,0.171340,4.829,9.935,...,14.2,16.2,22.8,28.5,459.3,167.3,43.386,69.262,25.876,income-group-3
3064,Waldo County,Maine,70.0,176.3,14.0,39705.048213,35.260000,0.200000,1.320,3.246,...,8.1,29.0,21.0,28.3,13.3,263.3,-0.283,45.859,46.142,income-group-3
3065,Perkins County,Nebraska,32.0,1106.9,9.0,2890.956726,311.315625,0.281250,3.940,1.868,...,8.2,20.7,26.2,24.5,6.4,349.9,73.484,84.767,11.283,income-group-4


<div class='exercise'><b>Exercise 4.6 [5 pts]: Aggregate data</b>
    
    
Write code in the cell below to group (and display) the data according to the 4 income groups. Also, while we will still keep the same columns (i.e, features), the values of each should now represent the __average__ value of all rows that were subsumed in the making of the aggregate income-group. Your resulting DataFrame should have just 4 rows (income-group-1, income-group-2, income-group-3, income-group-4). See example in the cell below.


Since every feature (except for `# total cases`, `# covid deaths`, and `population`) was already an average value corresponding to a particular __county__, when we aggregate our data by income groups, we are effectively taking an average of an average. Many counties are being aggregated for each income-group row. This approach isn't as accurate as possible; it would be more accurate if we re-adjusted every value so that it was truly an average that was based on the total __population__ of all counties that are subsumed within a given income-group row. That's okay, though. An average of averages will suffice for the purpose of this exercise. 
</div>

In [39]:
# EXAMPLE: If our `merged` DataFrame were
# COUNTY    INCOME GROUP    BACHELOR ... (other columns, too)
#   A            2             50
#   B            1             20
#   C            1             30
#   D            2             70
#   E            3             95

# it should become
# INCOME GROUP    BACHELOR ... (other columns, too)
#   1                25
#   2                60
#   3                95

# YOUR CODE HERE
merged.groupby(by='income group').agg(['mean'])
# END OF YOUR CODE HERE

,total covid cases,covid cases per 100k,covid deaths,population,covid deaths per 100k,case_fatality_rate,hispanic,minority,female,unemployed,income,nodegree,bachelor,inactivity,obesity,density,cancer,votergap,trump,clinton
,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
income group,,,,,,,,,,,,,,,,,,,,
income-group-1,887.150685,2619.673459,31.196918,31424.259807,72.584623,0.028281,9.313966,27.185592,50.091146,7.387329,33929.167808,22.453253,13.558048,30.507705,34.050856,91.363699,237.477625,27.111284,62.072111,34.960795
income-group-2,1669.326866,1730.351791,41.435821,77177.273092,40.634421,0.025455,9.595172,14.184955,49.972740,5.886716,41637.598507,16.440597,17.334925,27.312836,31.876119,151.710000,232.942944,37.397448,66.626127,29.228696
income-group-3,2519.157895,1637.670628,61.796265,122551.034245,35.225528,0.025097,9.663998,11.991611,50.027864,5.044992,48485.906621,13.418506,20.599491,25.136503,30.778268,400.207640,226.140823,32.018133,63.449908,31.431764
income-group-4,4748.193787,1453.014349,160.423077,257729.345317,38.773313,0.027235,10.571192,12.613022,50.199361,4.471598,62701.084320,10.616272,28.243935,22.055030,28.404882,399.271006,213.008321,20.590611,57.490584,36.900000


<div class='exercise'><b>Exercise 4.7 [2 pts]: Conclusions</b>
</div>

<div class='exercise'><b>Exercise 4.7: Q1 (1 of 2 pts)</b>
What are your conclusions/finding from this alternative view of the data? (2-4 sentences).
</div>

*Answer:* It's important to note that as we move up the income group quartiles the average population rises signficantly, which is likely due to many poor people living in the country's rural interior where their land and property is not as valuable as wealthier people living in urban and suburban areas. This population increase over the quartiles is likely the main reason that we see total covid cases and covid deaths increase as we move up through the quartiles. What's more important to observe is that on a per capita basis we see this trend reverse, where covid cases per 100k and covid deaths per 100k decline as we move up the income group quartiles, lending support to our hypothesis that poorer individuals and counties are more susceptible to covid infections and deaths.

<div class='exercise'><b>Exercise 4.7: Q2 (1 of 2 pts)</b>
What are some weaknesses from this view of the data? (2-4 sentences).
</div>

*Answer:* A key weakness of only looking at income group quartiles is neglecting the very salient differences in geography between the counties and the states that make up our dataset. Geography is critical confounding factor that's not examined by this view of the data. For the highest income group, for example, living in dense Seattle during the height of the crisis in March when many hospitals were packed (even if you're rich) meant you were much more likely to be infected by covid and die because the hospitals were packed, than in some less densely populated but also rich places like Palm Beach Island or Omaha, Nebraska. Moreover, political affiliation is largely uncorrelated with income, but an individual's political views or beliefs can drastically impact their personal behavior regarding the virus and subsequently their likelihood of being infected and dying from covid. Political affiliation is another cofounding factor that is a weakness of this view of the data.

## Moving Forward

In this homework assignment, we've focused on gathering, parsing, and exploring data. However, what if we wanted to *predict* some behavior of the data. For example, imagine one is curious how a particular county will respond to COVID. Or, imagine we looked at counties' COVID data on a weekly basis, one could be interested in predicting the upcoming week's behavior.

Alternatively, one could be interested in *inference*, whereby we are more concerned with trying to understand __why__ and __how__ a system behaves the way it does. We might wish to understand which factors most correlate and cause a certain event to happen. This could give us insights into where certain inequalities persist.

For both *prediction* and *inference*, our computational method of solving such a task is referred to as a model. For the remainder of CS109, we will spend significant focus on various models.
</div>

## Reflection

As a reminder, this is just **one** of the homework assignments in this course, the point of which is to assess your learning and to provide both you and us with an indication as to how aligned your knowledge and skills are with our learning objectives. To this end, we encourage you to reflect on your progress, strengths, and weaknesses and to make changes, if necessary, to accomplish your goals. Likewise, please reach out to the TFs and teaching staff if you need help. We want everyone to feel comfortable in being honest about these elements, with both herself/himself and us. For these purposes, we will ask you several times throughout the semester to complete an anonymous poll.